# Artur Fejklowicz - Bibliothekarin
Based on Kaggle 5 Day Intensive GenAI Course with Google - Day 3 Function Calling.

For my daughter Maja.

## Goals
1. Increase versatile of read books by a child. Child can say what it wants to read and this agent should recommend one book by title.
2. Allow to fast find the needed book in a bookcase like Ikea Kallax 3x3. Agent should be able to answer with x, y position of a shelf where book is stored in bookcase.

## High-level overview
1. Initialize SQLite database with book's metadata. Data collected manually by me for 11 books.
2. Define 3 Python functions, that will be used as a tools by LLM.
3. Naive RAG - index synthetic books descriptions with isbn as id. They are created from my prompt in Gemini 2.5 and here pasted. Embeddings model: text-embedding-004 from Google.
4. Naive RAG - query. Initialize Gemini Chat with 3 tools to be ble to authonomusly decide about their use. Sent user Query

## Setup

In [2]:
# # Remove unused conflicting packages
# # !pip uninstall -qqy jupyterlab
# !pip install -U -q "google-genai==1.7.0"

In [3]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

### Set up API key

In [4]:
import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("API Key not found.")

### Automated retry

In [5]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Create a local database

In [ ]:
%reload_ext sql
import sqlalchemy
print(sqlalchemy.__version__)

1.3.24


In [ ]:
%sql sqlite:///sample.db

'Connected: @sample.db'

Backwards compatibility of SQLite and Jupiter

In [ ]:
%config SqlMagic.style = '_DEPRECATED_MARKDOWN'

### Create the tables and insert some synthetic data.

In [9]:
%%sql
DROP TABLE IF EXISTS bookcase;
-- Create the 'bookcase' table
CREATE TABLE IF NOT EXISTS bookcase (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    isbn VARCHAR(17) UNIQUE,
    title VARCHAR(255) NOT NULL,
    author VARCHAR(255) NOT NULL,
    language VARCHAR(255),
    cover_color VARCHAR(255),
    cover_height_mm INTEGER,
    release_year INTEGER,
    number_of_pages INTEGER,
    shelf_x INTEGER,
    shelf_y INTEGER,
    CONSTRAINT unique_isbn UNIQUE (isbn)
  );
-- INSERT sample data
INSERT INTO bookcase (isbn, title, author, language, cover_color, cover_height_mm, release_year, number_of_pages, shelf_x, shelf_y) VALUES
('978-83-10-12960-4', 'Brzechwa dzieciom. Dzieła wszystkie: Bajki',       'Jan Brzechwa',                       'polish',   'yellow',   250, 2018, 352, 1, 1),
('978-83-240-3926-5', 'Wierszyki domowe',                                 'Michał Rusinek',                     'polish',   'red',      213, 2016, 184, 1, 2),
('978-1-4071-9557-5', 'The Wonky Donkey',                                 'Craig Smith',                        'english',  'blue',     250, 2018, 24,  1, 3),
('978-83-8073-725-9', 'Wiewiórki, które nie chciały się dzielić',         'Rachel Bright, Jim Field',           'polish',   'brown',    330, 2020, 32,  2, 1),
('978-3-03869-022-1', 'Ganz schön Schweiz. Ein cartoon survival guide.',  'Sergio J. Lievano, Wolfgang Koydl',  'german',   'red',      250, 2017, 160, 2, 2),
('978-83-8154-402-3', 'Tuż przed gwiazdką',                               'Barbara Supeł, Marta Koshulinska',   'polish',   'red',      250, 2019, 48,  2, 3),
('978-83-8141-546-0', 'Walizka pełna marzeń',                             'Bardijewska Liliana',                'polish',   'red',      150, 2022, 112, 3, 1),
('978-83-8057-731-2', 'Dziewczynki latają wysoko',                        'Raquel Diaz Reguera',                'polish',   'yellow',   230, 2018, 48,  3, 2),
('978-3-551-28005-3', 'Die Eiskönigin',                                   'Disney',                             'german',   'blue',     285, 2018, 80,  1, 1),
('978-3-551-51841-5', 'Das Neinhorn',                                     'Marc-Uwe Kling, Astrid Henn',        'german',   'blue',     276, 2019, 48,  1, 1),
('978-83-240-9399-1', 'Odlotowe gołębie łapią bandytów',                  'Andrew McDonald, Ben Wood',          'polish',   'yellow',   150, 2023, 112, 2, 1)
;


 * sqlite:///sample.db
Done.
Done.
11 rows affected.


[]

In [10]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///sample.db
Done.


name
sqlite_sequence
bookcase


In [11]:
%sql SELECT * FROM bookcase;

 * sqlite:///sample.db
Done.


book_id,isbn,title,author,language,cover_color,cover_height_mm,release_year,number_of_pages,shelf_x,shelf_y
1,978-83-10-12960-4,Brzechwa dzieciom. Dzieła wszystkie: Bajki,Jan Brzechwa,polish,yellow,250,2018,352,1,1
2,978-83-240-3926-5,Wierszyki domowe,Michał Rusinek,polish,red,213,2016,184,1,2
3,978-1-4071-9557-5,The Wonky Donkey,Craig Smith,english,blue,250,2018,24,1,3
4,978-83-8073-725-9,"Wiewiórki, które nie chciały się dzielić","Rachel Bright, Jim Field",polish,brown,330,2020,32,2,1
5,978-3-03869-022-1,Ganz schön Schweiz. Ein cartoon survival guide.,"Sergio J. Lievano, Wolfgang Koydl",german,red,250,2017,160,2,2
6,978-83-8154-402-3,Tuż przed gwiazdką,"Barbara Supeł, Marta Koshulinska",polish,red,250,2019,48,2,3
7,978-83-8141-546-0,Walizka pełna marzeń,Bardijewska Liliana,polish,red,150,2022,112,3,1
8,978-83-8057-731-2,Dziewczynki latają wysoko,Raquel Diaz Reguera,polish,yellow,230,2018,48,3,2
9,978-3-551-28005-3,Die Eiskönigin,Disney,german,blue,285,2018,80,1,1
10,978-3-551-51841-5,Das Neinhorn,"Marc-Uwe Kling, Astrid Henn",german,blue,276,2019,48,1,1


## Define database functions

Function calling with Gemini API's Python SDK can be implemented by defining [an OpenAPI schema](https://ai.google.dev/api/caching#Schema) that is passed to the model. You can also define Python functions and let the SDK inspect them to automatically define the schema. In this latter case, it's important that the functions are type annotated and have accurate docstrings that describe what the functions do - the model has no insight into the function body, so the docs function as the interface.

By providing three key pieces of functionality - listing tables, describing a table, and executing a query - the LLM (much like a human user) will have the basic tools needed to understand and interrogate the database.

Start with a database connection that will be used across all of the functions.

In [12]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [13]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['sqlite_sequence', 'bookcase']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [14]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("bookcase")

 - DB CALL: describe_table(bookcase)


[('book_id', 'INTEGER'),
 ('isbn', 'VARCHAR(17)'),
 ('title', 'VARCHAR(255)'),
 ('author', 'VARCHAR(255)'),
 ('language', 'VARCHAR(255)'),
 ('cover_color', 'VARCHAR(255)'),
 ('cover_height_mm', 'INTEGER'),
 ('release_year', 'INTEGER'),
 ('number_of_pages', 'INTEGER'),
 ('shelf_x', 'INTEGER'),
 ('shelf_y', 'INTEGER')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [ ]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall() 


execute_query("SELECT * FROM bookcase LIMIT 1")

 - DB CALL: execute_query(select * from bookcase)


[(1,
  '978-83-10-12960-4',
  'Brzechwa dzieciom. Dzieła wszystkie: Bajki',
  'Jan Brzechwa',
  'polish',
  'yellow',
  250,
  2018,
  352,
  1,
  1),
 (2,
  '978-83-240-3926-5',
  'Wierszyki domowe',
  'Michał Rusinek',
  'polish',
  'red',
  213,
  2016,
  184,
  1,
  2),
 (3,
  '978-1-4071-9557-5',
  'The Wonky Donkey',
  'Craig Smith',
  'english',
  'blue',
  250,
  2018,
  24,
  1,
  3),
 (4,
  '978-83-8073-725-9',
  'Wiewiórki, które nie chciały się dzielić',
  'Rachel Bright, Jim Field',
  'polish',
  'brown',
  330,
  2020,
  32,
  2,
  1),
 (5,
  '978-3-03869-022-1',
  'Ganz schön Schweiz. Ein cartoon survival guide.',
  'Sergio J. Lievano, Wolfgang Koydl',
  'german',
  'red',
  250,
  2017,
  160,
  2,
  2),
 (6,
  '978-83-8154-402-3',
  'Tuż przed gwiazdką',
  'Barbara Supeł, Marta Koshulinska',
  'polish',
  'red',
  250,
  2019,
  48,
  2,
  3),
 (7,
  '978-83-8141-546-0',
  'Walizka pełna marzeń',
  'Bardijewska Liliana',
  'polish',
  'red',
  150,
  2022,
  112,
  3,


## Naive RAG for books descriptions

First install Chroma

In [24]:
!pip install -qU "chromadb==0.6.3"

In [28]:
SYNTH_DESC1 = r"""Ta książka to zbiór najbardziej znanych i lubianych bajek napisanych przez Jana Brzechwę. Znajdziesz w niej opowieści o zwierzętach, ludziach i niezwykłych zdarzeniach, pełne humoru i mądrości. Poznasz historię Kaczki Dziwaczki, która wszystko robiła na odwrót i pływała w rondlu. Dowiesz się, dlaczego Stonoga spóźniła się na bal i jak liczyła swoje nóżki. Odkryjesz też przygody sprytnej Pchły Szachrajki, która potrafiła oszukać nawet lwa. Przeczytasz o Sójce, która wybierała się za morze, ale nigdy tam nie dotarła. W każdej bajce kryje się morał, który uczy czegoś ważnego o życiu. Bohaterowie tych opowieści, często zabawni i trochę dziwni, przeżywają ciekawe przygody. To wierszowane historyjki, które łatwo wpadają w ucho i bawią czytelników w każdym wieku. Wyrusz w podróż do świata fantazji, śmiechu i ważnych lekcji z wierszami Brzechwy."""
SYNTH_DESC2 = r"""Ta książka zawiera zbiór zabawnych wierszyków o przedmiotach, które można znaleźć w każdym domu. Autor w dowcipny sposób opisuje codzienne rzeczy, nadając im niezwykłe cechy. Dowiesz się, co myśli sobie stary, skrzypiący kaloryfer albo jakie przygody przeżywa pilot od telewizora. Przeczytasz o sekretnym życiu pralki podczas wirowania i o tym, co dzieje się w kuchennym zlewie po zmroku. Wierszyki pokazują, że nawet najbardziej zwyczajne przedmioty mogą ożyć w naszej wyobraźni. Każdy wierszyk to krótka, rymowana historyjka pełna zaskakujących pomysłów i humoru. Dzięki tej książce spojrzysz na swoje mieszkanie zupełnie inaczej, odkrywając magię w codzienności. To lektura, która udowadnia, że świat wokół nas jest pełen ciekawych opowieści. Znajdziesz tu mnóstwo śmiechu i inspiracji do własnych obserwacji. Przekonaj się, jak fascynujący może być Twój własny dom dzięki tym wesołym rymowankom."""
SYNTH_DESC3 = r"""This book tells the funny story of a donkey walking down the road. He isn't just any ordinary donkey; he's quite unique and very silly. As the story unfolds, you learn more and more amusing things about him. First, you discover that he only has three legs, making him a wonky donkey. Then, you find out he also has only one eye, adding another detail to his description. Each page adds a new, funny characteristic to the growing list of his quirks. The descriptions get longer and more hilarious with every turn of the page. It's a story built on repetition and adding funny adjectives to describe the donkey. You'll surely giggle trying to keep up with all his silly features. Join the fun and meet the progressively sillier, stinky, lanky, honky-tonky, winky, wonky donkey!"""
SYNTH_DESC4 = r"""Ta książka opowiada historię dwóch bardzo chciwych wiewiórek, Cyryla i Błażeja. Pewnego jesiennego dnia obie wiewiórki znalazły ostatnią, idealną szyszkę na zimę. Żadna z nich nie miała najmniejszego zamiaru podzielić się swoim znaleziskiem z drugą. Rozpoczęła się szalona pogoń przez las, pełna podstępów i sztuczek, aby zdobyć szyszkę tylko dla siebie. Ich zacięta rywalizacja doprowadziła do wielu zabawnych, ale też niebezpiecznych sytuacji. W końcu ich kłótnia i walka o szyszkę wpędziły je w poważne tarapaty, z których mogły wyjść tylko razem. W obliczu niebezpieczeństwa musiały nauczyć się, że współpraca jest ważniejsza niż posiadanie czegoś na własność. Odkryły, że przyjaźń i dzielenie się przynoszą o wiele więcej radości niż chciwość. To ciepła i mądra opowieść o tym, jak ważne jest bycie dobrym dla innych. Zobacz, jak Cyryl i Błażej zrozumieli swój błąd i nauczyli się prawdziwej wartości dzielenia się."""
SYNTH_DESC5 = r"""Dieses Buch ist ein lustiger Comic-Ratgeber über die Schweiz und ihre besonderen Eigenschaften. Es zeigt mit vielen witzigen Bildern, was typisch für die Schweizer Kultur und das Leben hier ist. Du lernst auf humorvolle Weise die Gewohnheiten der Menschen in der Schweiz kennen. Das Buch erklärt zum Beispiel mit Cartoons, warum Pünktlichkeit hier so wichtig ist. Es zeigt auch, wie man sich in den Bergen richtig verhält oder was der "Röstigraben" bedeutet. Viele bunte Zeichnungen machen das Entdecken der Schweizer Besonderheiten zu einem grossen Spass. Kleine Alltagssituationen werden dargestellt, die manchmal etwas seltsam oder lustig wirken können. Du erfährst auf leichte und unterhaltsame Art mehr über das Land und seine Traditionen. Die Cartoons helfen dir, die Schweiz und ihre Leute mit einem Augenzwinkern besser zu verstehen. Entdecke die Schweiz auf eine ganz neue, witzige Art mit diesem Cartoon-Führer!"""
SYNTH_DESC6 = r"""Ta książka opowiada o wyjątkowym, magicznym czasie tuż przed Świętami Bożego Narodzenia. Poznajemy rodzinę, która z wielką radością i zaangażowaniem przygotowuje się do nadchodzącej Gwiazdki. Dzieci z niecierpliwością wypatrują pierwszej gwiazdki na niebie i marzą o prezentach pod choinką. W całym domu unosi się cudowny zapach pieczonych pierników i świątecznych potraw. Za oknem delikatnie prószy śnieg, tworząc zimową atmosferę oczekiwania. Książka opisuje codzienne przygotowania, takie jak wspólne pieczenie ciasteczek, robienie ozdób czy ubieranie choinki. Bohaterowie przeżywają małe, przedświąteczne przygody i doświadczają miłych niespodzianek. To ciepła opowieść o rodzinnych więziach, radości oczekiwania i niepowtarzalnej magii świąt. Poczujesz atmosferę przygotowań i podekscytowanie związane ze zbliżającą się Wigilią. Historia przypomina, jak ważne są wspólnie spędzane chwile w tym niezwykłym okresie roku."""
SYNTH_DESC7 = r"""Ta książka zabierze Cię w niezwykłą podróż do krainy wyobraźni i wielkich marzeń. Opowiada historię kogoś, kto posiada tajemniczą, starą walizkę. Jednak ta walizka nie jest wypełniona zwykłymi rzeczami, jak ubrania czy pamiątki z podróży. W jej wnętrzu kryją się najpiękniejsze i najskrytsze marzenia, pragnienia oraz nadzieje. Bohaterowie opowieści przeżywają przygody, które pomagają im odkryć, czego tak naprawdę pragną w życiu. Historia pokazuje, jak ważne jest, aby nigdy nie rezygnować ze swoich marzeń i dążyć do ich spełnienia. Książka uczy odwagi w podążaniu własną ścieżką i wiary we własne możliwości. To piękna opowieść o tym, że każdy z nas nosi w sobie potencjał do realizacji swoich pragnień. Dowiesz się, że siła wyobraźni potrafi zaprowadzić nas w niesamowite miejsca. Otwórz tę symboliczną walizkę razem z bohaterami i odkryj potęgę swoich własnych marzeń."""
SYNTH_DESC8 = r"""Ta książka opowiada inspirujące historie dziewczynek, które mają odwagę marzyć i realizować swoje pasje. Każda z bohaterek jest inna, ma swoje zainteresowania i talenty, ale wszystkie łączy pragnienie sięgania wysoko. Poznajemy dziewczynki, które nie boją się być sobą i robić tego, co naprawdę kochają, nawet jeśli jest to nietypowe. Niektóre marzą o lataniu samolotem, inne chcą zostać naukowczyniami, odkrywczyniami czy wielkimi artystkami. Książka pokazuje z mocą, że dziewczynki mogą osiągnąć absolutnie wszystko, czego tylko zapragną. Bohaterki udowadniają, że nie ma rzeczy niemożliwych, jeśli mocno w siebie wierzymy i ciężko pracujemy. To wspaniała opowieść o sile, determinacji, odwadze i przełamywaniu krzywdzących stereotypów. Inspiruje każdą młodą czytelniczkę do tego, by nie bała się marzyć odważnie i dążyć do realizacji swoich celów. Przekonasz się, że każda dziewczynka ma w sobie niezwykłą moc do czynienia wielkich rzeczy."""
SYNTH_DESC9 = r"""Dieses Buch erzählt die bekannte und beliebte Geschichte der zwei königlichen Schwestern Elsa und Anna. Sie leben glücklich als Prinzessinnen im wunderschönen Königreich Arendelle. Die ältere Schwester Elsa besitzt eine geheime und mächtige Zauberkraft: Sie kann Eis und Schnee erschaffen. Als ihre Kräfte bei ihrer Krönungsfeier versehentlich enthüllt werden, flieht Elsa aus Angst in die einsamen Berge. Dabei stürzt sie Arendelle ungewollt in einen ewig dauernden, eisigen Winter. Ihre mutige und optimistische Schwester Anna macht sich auf eine gefährliche Reise, um Elsa zu finden und den Winterzauber zu brechen. Unterwegs trifft Anna neue, lustige Freunde wie den liebenswerten Schneemann Olaf und den hilfsbereiten Eislieferanten Kristoff mit seinem Rentier Sven. Gemeinsam erleben sie spannende und magische Abenteuer im frostigen Land. Es ist eine herzerwärmende Geschichte über die unzertrennliche Verbindung zwischen Schwestern und die wahre Kraft der Liebe."""
SYNTH_DESC10 = r"""Dieses Buch erzählt die sehr lustige Geschichte eines kleinen, aber ganz besonderen Einhorns. Es wurde im bunten Herzwald geboren, doch es ist überhaupt nicht süss und lieb, wie man es von Einhörnern erwartet. Dieses störrische Einhorn sagt zu allem und jedem einfach immer nur "Nein!". Wegen seiner ständigen Ablehnung wird es von allen bald nur noch das "Neinhorn" genannt. Eines Tages hat das Neinhorn genug von der übertrieben zuckersüssen Einhornwelt und beschliesst, wegzulaufen. Auf seinem Weg trifft es andere Tiere, die auch ein bisschen anders und speziell sind. Da ist zum Beispiel ein Waschbär, der nie richtig zuhört, und ein Hund, dem alles völlig egal ist, sowie eine Prinzessin, die ständig widerspricht. Zusammen bilden sie ein urkomisches Team von Querköpfen, Grummlern und Trotzern. Es ist eine äusserst witzige Geschichte über schlechte Laune, das Anderssein und das Finden von Freunden, die einen so akzeptieren, wie man ist."""
SYNTH_DESC11 = r"""Ta książka opowiada o niezwykłej grupie gołębi, które potajemnie pracują jako tajni agenci. To nie są zwyczajne ptaki, które spotykasz na ulicy – one walczą z przestępczością i łapią groźnych bandytów. Zespół tych odlotowych gołębi rozwiązuje skomplikowane zagadki kryminalne i tropi złoczyńców w całym mieście. Każdy gołąb w drużynie posiada inne, wyjątkowe umiejętności i gadżety, które pomagają im w trudnych misjach. Razem muszą powstrzymać przebiegłych przestępców i udaremnić ich nikczemne plany, ratując miasto przed kłopotami. Podczas swoich tajnych operacji przeżywają mnóstwo zabawnych i pełnych napięcia przygód. Książka jest pełna zaskakującego humoru, dynamicznej akcji i nieoczekiwanych zwrotów wydarzeń, które trzymają w napięciu. Dowiesz się, jak sprytne, odważne i niezwykle pomysłowe potrafią być te niezwykłe gołębie-detektywi. To wciągająca historia o sile przyjaźni, niezawodnej współpracy w zespole i odwiecznej walce dobra ze złem. Dołącz do ekipy odlotowych gołębi i przeżyj razem z nimi niesamowite śledztwo pełne śmiechu i emocji!"""
synthetic_descriptions = [SYNTH_DESC1, SYNTH_DESC2, SYNTH_DESC3, SYNTH_DESC4, SYNTH_DESC5, SYNTH_DESC6, SYNTH_DESC7, SYNTH_DESC8, SYNTH_DESC9, SYNTH_DESC10, SYNTH_DESC11]

## Initialize Gemini API Client
This is used in function, that creates embeddings and later to initialize chat. 

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

## Creating the embedding database with ChromaDB

Create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. In this task, you are implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, you will use `retrieval_query` for the *query* embeddings. Check out the [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) for the full list of supported tasks.

Key words: Documents are the items that are in the database. They are inserted first, and later retrieved. Queries are the textual search terms and can be simple keywords or textual descriptions of the desired documents.

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

In [ ]:
import chromadb

DB_NAME = "books_descriptions"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

isbns = [isbn[0] for isbn in execute_query("SELECT isbn FROM bookcase ORDER BY book_id")]
vector_store.add(documents=synthetic_descriptions, ids=isbns)

Confirm that the data was inserted by looking at the database.

In [174]:
len(vector_store.peek(1)["embeddings"][0])

768

## Function calling
Passing created Python functions as tools, while creating Gemini session using Gemini API.

In [ ]:
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot as Minecraft female Librarian, that help manage girl's bookcase.
You answers questions using book descriptions from the reference passages included below. In passages are only book's descriptions. In database is more info about a book.
If the passage is irrelevant to the answer, you may ignore it.
You can interact with an SQL database. Please before each execute_query understand schema, so you know how to build SQL.
You will take the users questions and turn them into SQL queries using the tools available in order to provide additional information about the book.
You can join passage to data in SQL database, based on this rule: PASSAGE_ID = bookcase.isbn.
Once you have the information you need, you will answer the user's question in english language, using the data returned.
You will talk to a 8 year old girl, so you use apropriate language form with kindeness and sympathy.

Use tools provided
- list_tables to see what tables are present, 
- describe_table to understand the schema, 
- execute_query to issue an SQL SELECT query. 
"""

client = genai.Client(api_key=GOOGLE_API_KEY)
# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

## Retrieval: Find relevant documents

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False
query = "Where is the book about Switzerland in my library?"

result = vector_store.query(query_texts=[query], n_results=2)
[all_passages] = result["documents"]
[all_passage_ids] = result["ids"]
Markdown(all_passages[0])

Dieses Buch ist ein lustiger Comic-Ratgeber über die Schweiz und ihre besonderen Eigenschaften. Es zeigt mit vielen witzigen Bildern, was typisch für die Schweizer Kultur und das Leben hier ist. Du lernst auf humorvolle Weise die Gewohnheiten der Menschen in der Schweiz kennen. Das Buch erklärt zum Beispiel mit Cartoons, warum Pünktlichkeit hier so wichtig ist. Es zeigt auch, wie man sich in den Bergen richtig verhält oder was der "Röstigraben" bedeutet. Viele bunte Zeichnungen machen das Entdecken der Schweizer Besonderheiten zu einem grossen Spass. Kleine Alltagssituationen werden dargestellt, die manchmal etwas seltsam oder lustig wirken können. Du erfährst auf leichte und unterhaltsame Art mehr über das Land und seine Traditionen. Die Cartoons helfen dir, die Schweiz und ihre Leute mit einem Augenzwinkern besser zu verstehen. Entdecke die Schweiz auf eine ganz neue, witzige Art mit diesem Cartoon-Führer!

Create user message by filling prompt template with parameters and send it to Gemini

In [166]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""QUESTION: {query_oneline}\n\n"""

# Add the retrieved documents to the prompt.
# for passage in all_passages:
for idx, passage in enumerate(all_passages):
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE_ID: {all_passage_ids[idx]}\nPASSAGE: {passage_oneline}\n"

print(prompt)
resp = chat.send_message(prompt)
print(f"\n{resp.text}")

QUESTION: Where in my library is the book about Switzerland?

PASSAGE_ID: 978-3-03869-022-1
PASSAGE: Dieses Buch ist ein lustiger Comic-Ratgeber über die Schweiz und ihre besonderen Eigenschaften. Es zeigt mit vielen witzigen Bildern, was typisch für die Schweizer Kultur und das Leben hier ist. Du lernst auf humorvolle Weise die Gewohnheiten der Menschen in der Schweiz kennen. Das Buch erklärt zum Beispiel mit Cartoons, warum Pünktlichkeit hier so wichtig ist. Es zeigt auch, wie man sich in den Bergen richtig verhält oder was der "Röstigraben" bedeutet. Viele bunte Zeichnungen machen das Entdecken der Schweizer Besonderheiten zu einem grossen Spass. Kleine Alltagssituationen werden dargestellt, die manchmal etwas seltsam oder lustig wirken können. Du erfährst auf leichte und unterhaltsame Art mehr über das Land und seine Traditionen. Die Cartoons helfen dir, die Schweiz und ihre Leute mit einem Augenzwinkern besser zu verstehen. Entdecke die Schweiz auf eine ganz neue, witzige Art mit 